<a href="https://colab.research.google.com/github/MinZhenTay/Recipe-Recommendation/blob/main/Web_Scrapping_%26_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web Scrapping**

In [1]:
# from selenium import webdriver
# Selenium WebDriver acts as web framework to execute cross-browser tests
from   bs4 import BeautifulSoup
import pandas as pd

In [2]:
import urllib
import urllib.request

In [3]:
from urllib.request import Request, urlopen
headers = {'User-Agent': 'Mozilla/5.0'}

In [4]:
web='https://www.allrecipes.com/recipes-a-z-6735880'

In [5]:
request = Request(web, headers=headers)
 # url = urllib.request.urlopen(web, headers={'User-Agent': 'Mozilla/5.0'})
content = urlopen(request).read()
soup = BeautifulSoup(content, 'lxml')

In [6]:
# Create empty list for link, link name and reciep link
linkname=[]
link=[]
recipelink=[]

In [7]:
for a in soup.findAll('li', attrs={'comp link-list__item'}):
  # print(a)
  l=a.find('a',href=True, attrs={'link-list__link type--dog-bold type--dog-link'})
  link.append(l.get('href'))

In [8]:
# Scrap all the recipie links
for web in link:
  request = Request(web, headers=headers)
 # url = urllib.request.urlopen(web, headers={'User-Agent': 'Mozilla/5.0'})
  content = urlopen(request).read()
  soup = BeautifulSoup(content, 'lxml')
  for a in soup.findAll('a',href=True, attrs={'comp mntl-card-list-items mntl-document-card mntl-card card card--no-image'}):
    recipelink.append(a.get('href'))
  


In [9]:
# Remove links without recipe 
import re
to_remove = ['gallery','article']

result = [i for i in recipelink if not re.search("|".join(to_remove), i)]
# print(result)

In [10]:
# Create empty dataframe for title, description, ingredient and cooking directions
title=[]
description=[]
ingredient=[]
directions=[]

In [11]:
# Create empty list for nutritional values
nutrient=[]

In [13]:
# Code to obtain nutrient data
for webrec in result:
  nutri=[]
  # print(webrec)
  request = Request(webrec, headers=headers)
 # url = urllib.request.urlopen(web, headers={'User-Agent': 'Mozilla/5.0'})
  content = urlopen(request).read()
  soup = BeautifulSoup(content, 'lxml')
  for a in soup.findAll('tr', attrs={'mntl-nutrition-facts-summary__table-row'}):

    dictionary = {}
    cal=a.find('td', attrs={'mntl-nutrition-facts-summary__table-cell type--dog'})
    value=a.find('td', attrs={'mntl-nutrition-facts-summary__table-cell type--dog-bold'}) 
    # print(a)
   

    if(cal==None):
      # print(cal.text)
      dictionary['Calories'] = value.text
    else:
      dictionary[cal.text] = value.text
    nutri.append(dictionary)
  nutrient.append(nutri)  


In [14]:
import numpy as np

# Save the nutrient data into dataframe
nutrient = np.array(nutrient )
nutrient = pd.DataFrame(nutrient,columns = ['nutrient'])

<ipython-input-14-f592eeab3a79>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  nutrient = np.array(nutrient )


In [18]:
# Code to scrap description
for webrec in result:
  ing=[]
  rcp=[]
  
  request = Request(webrec, headers=headers)
 # url = urllib.request.urlopen(web, headers={'User-Agent': 'Mozilla/5.0'})
  content = urlopen(request).read()
  soup = BeautifulSoup(content, 'lxml')
  # Code to scrap recipe title
  h1tag=soup.find('h1',attrs={'comp type--lion article-heading mntl-text-block'})
  title.append(h1tag.text)
  
  # Code to scrap description name
  ptag=soup.find('p',attrs={'comp type--dog article-subheading'})
  description.append(ptag.text)
  # Code to scrap description
  for a in soup.findAll('li', attrs={'mntl-structured-ingredients__list-item '}):
    ingre=a.find('p')
    ing.append(ingre.text)
  ingredient.append(ing)
  # Code to scrap cooking directions
  for a in soup.findAll('li', attrs={'comp mntl-sc-block-group--LI mntl-sc-block mntl-sc-block-startgroup'}):
    step=a.find('p')
  # print(step.text)
    rcp.append(step.text)
  directions.append(rcp)

In [19]:
# Convert into array
ingredient = np.array(ingredient )
title = np.array(title )
description = np.array(description )
directions = np.array(directions )

<ipython-input-19-fcf4df9fb6c7>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ingredient = np.array(ingredient )
<ipython-input-19-fcf4df9fb6c7>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  directions = np.array(directions )


In [20]:
# Converts data into pandas dataframe
ingredient_df = pd.DataFrame(ingredient,columns = ['Ingredient'])

In [22]:
import numpy as np
import pandas as pd

# Convert all data into dataframe
ingredient_df = pd.DataFrame(ingredient,columns = ['Ingredient'])
directions_df = pd.DataFrame(directions,columns = ['directions'])
description_df = pd.DataFrame(description,columns = ['description'])
title_df = pd.DataFrame(title,columns = ['title'])

# Concatinate all dataframes
frames = [title_df, ingredient_df, directions_df, description_df, nutrient]

df = pd.concat(frames, axis=1)
df

,title,Ingredient,directions,description,nutrient
0,\nAir Fryer Halloumi Cheese,"[8 ounces halloumi cheese, 2 teaspoons olive o...",[\nPreheat the air fryer to 360 degrees F (180...,\nThe first time I had halloumi was in a pub i...,"[{'Calories': '66'}, {'Fat': '6g '}, {'Carbs':..."
1,\nAir Fryer Ranch Chicken Bites,"[1 pound boneless, skinless chicken thighs, tr...","[\nCombine chicken, avocado oil, white vinegar...","\nIncredibly tender and flavorful, and the kid...","[{'Calories': '328'}, {'Fat': '26g '}, {'Carbs..."
2,\nAir Fryer Frozen Chicken Strips,"[25 ounces frozen cooked, breaded crispy chick...",[\nSet the air fryer temperature to 380 degree...,\nClassic finger food which we enjoy with a di...,"[{'Calories': '106'}, {'Fat': '2g '}, {'Carbs'..."
3,\nAir Fryer Spicy Onion Rings,"[2 large sweet onions, sliced 1/2 inch thick, ...","[\nWhisk together buttermilk, egg, flour, chil...","\nCrispy and crunchy on the outside, tender on...","[{'Calories': '230'}, {'Fat': '4g '}, {'Carbs'..."
4,\nAir Fryer BBQ Chicken Drummies,"[2 pounds chicken drummies, 1 ½ tablespoons ol...",[\nPreheat an air fryer to 360 degrees F (180 ...,\nThese drummies are perfectly crisp on the ou...,"[{'Calories': '483'}, {'Fat': '25g '}, {'Carbs..."
...,...,...,...,...,...
53270,\nLemon Zucchini Muffins,"[1 ¾ cups all-purpose flour, ¾ cup white sugar...",[\nPreheat oven to 400 degrees F (200 degrees ...,\nI used to love the low-fat lemon zucchini mu...,NaN
53271,\nZucchini Pineapple Bread,"[3 cups all-purpose flour, 2 teaspoons baking ...",[\nPreheat an oven to 325 degrees F (165 degre...,\nThe pineapple makes this bread extra moist w...,NaN
53272,\nWhole Wheat Zucchini Herb Bread,"[½ cup water, 2 teaspoons honey, 1 tablespoon ...",[\nPlace ingredients in the pan of the bread m...,\nThis bread is very light and rises high.,NaN
53273,\nVegan Zucchini Bread,"[3 cups all-purpose flour, 1 teaspoon salt, 1 ...",[\nPreheat an oven to 325 degrees F (165 degre...,"\nThis yummy, moist, rich zucchini bread is da...",NaN


In [23]:
# Save the dataframe into csv file for later use 
df.to_csv('RecipeData.csv')

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# Upload the downloaded dataset

import pandas as pd
df1=pd.read_csv('/content/RecipeData.csv')

In [26]:
df1

,Unnamed: 0,title,Ingredient,directions,description,nutrient
0,0,\nAir Fryer Halloumi Cheese,"['8 ounces halloumi cheese', '2 teaspoons oliv...",['\nPreheat the air fryer to 360 degrees F (18...,\nThe first time I had halloumi was in a pub i...,"[{'Calories': '66'}, {'Fat': '6g '}, {'Carbs':..."
1,1,\nAir Fryer Ranch Chicken Bites,"['1 pound boneless, skinless chicken thighs, t...","['\nCombine chicken, avocado oil, white vinega...","\nIncredibly tender and flavorful, and the kid...","[{'Calories': '328'}, {'Fat': '26g '}, {'Carbs..."
2,2,\nAir Fryer Frozen Chicken Strips,"['25 ounces frozen cooked, breaded crispy chic...",['\nSet the air fryer temperature to 380 degre...,\nClassic finger food which we enjoy with a di...,"[{'Calories': '106'}, {'Fat': '2g '}, {'Carbs'..."
3,3,\nAir Fryer Spicy Onion Rings,"['2 large sweet onions, sliced 1/2 inch thick'...","['\nWhisk together buttermilk, egg, flour, chi...","\nCrispy and crunchy on the outside, tender on...","[{'Calories': '230'}, {'Fat': '4g '}, {'Carbs'..."
4,4,\nAir Fryer BBQ Chicken Drummies,"['2 pounds chicken drummies', '1 ½ tablespoons...",['\nPreheat an air fryer to 360 degrees F (180...,\nThese drummies are perfectly crisp on the ou...,"[{'Calories': '483'}, {'Fat': '25g '}, {'Carbs..."
...,...,...,...,...,...,...
53270,53270,\nLemon Zucchini Muffins,"['1 ¾ cups all-purpose flour', '¾ cup white su...",['\nPreheat oven to 400 degrees F (200 degrees...,\nI used to love the low-fat lemon zucchini mu...,NaN
53271,53271,\nZucchini Pineapple Bread,"['3 cups all-purpose flour', '2 teaspoons baki...",['\nPreheat an oven to 325 degrees F (165 degr...,\nThe pineapple makes this bread extra moist w...,NaN
53272,53272,\nWhole Wheat Zucchini Herb Bread,"['½ cup water', '2 teaspoons honey', '1 tables...",['\nPlace ingredients in the pan of the bread ...,\nThis bread is very light and rises high.,NaN
53273,53273,\nVegan Zucchini Bread,"['3 cups all-purpose flour', '1 teaspoon salt'...",['\nPreheat an oven to 325 degrees F (165 degr...,"\nThis yummy, moist, rich zucchini bread is da...",NaN


In [27]:
# Remove 'description' column from the dataset
df1=df1.drop(['description'], axis=1)

In [28]:
# Check the latest number of columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53275 entries, 0 to 53274
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53275 non-null  int64 
 1   title       53275 non-null  object
 2   Ingredient  53275 non-null  object
 3   directions  53275 non-null  object
 4   nutrient    18927 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.0+ MB


In [29]:
# Remove 'Unnamed: 0' column from the dataset
df1=df1.drop(['Unnamed: 0'], axis=1)

In [30]:
# Check the latest number of columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53275 entries, 0 to 53274
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       53275 non-null  object
 1   Ingredient  53275 non-null  object
 2   directions  53275 non-null  object
 3   nutrient    18927 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [31]:
# Rename the current columns so that the column names are same as the Kaggle dataset
df1.rename(columns = {'title':'recipe_name','Ingredient':'ingredients','directions':'cooking_directions','nutrient':'nutritions'}, inplace = True)

In [32]:
# Check for the updated column name
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53275 entries, 0 to 53274
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recipe_name         53275 non-null  object
 1   ingredients         53275 non-null  object
 2   cooking_directions  53275 non-null  object
 3   nutritions          18927 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [33]:
df1

,recipe_name,ingredients,cooking_directions,nutritions
0,\nAir Fryer Halloumi Cheese,"['8 ounces halloumi cheese', '2 teaspoons oliv...",['\nPreheat the air fryer to 360 degrees F (18...,"[{'Calories': '66'}, {'Fat': '6g '}, {'Carbs':..."
1,\nAir Fryer Ranch Chicken Bites,"['1 pound boneless, skinless chicken thighs, t...","['\nCombine chicken, avocado oil, white vinega...","[{'Calories': '328'}, {'Fat': '26g '}, {'Carbs..."
2,\nAir Fryer Frozen Chicken Strips,"['25 ounces frozen cooked, breaded crispy chic...",['\nSet the air fryer temperature to 380 degre...,"[{'Calories': '106'}, {'Fat': '2g '}, {'Carbs'..."
3,\nAir Fryer Spicy Onion Rings,"['2 large sweet onions, sliced 1/2 inch thick'...","['\nWhisk together buttermilk, egg, flour, chi...","[{'Calories': '230'}, {'Fat': '4g '}, {'Carbs'..."
4,\nAir Fryer BBQ Chicken Drummies,"['2 pounds chicken drummies', '1 ½ tablespoons...",['\nPreheat an air fryer to 360 degrees F (180...,"[{'Calories': '483'}, {'Fat': '25g '}, {'Carbs..."
...,...,...,...,...
53270,\nLemon Zucchini Muffins,"['1 ¾ cups all-purpose flour', '¾ cup white su...",['\nPreheat oven to 400 degrees F (200 degrees...,NaN
53271,\nZucchini Pineapple Bread,"['3 cups all-purpose flour', '2 teaspoons baki...",['\nPreheat an oven to 325 degrees F (165 degr...,NaN
53272,\nWhole Wheat Zucchini Herb Bread,"['½ cup water', '2 teaspoons honey', '1 tables...",['\nPlace ingredients in the pan of the bread ...,NaN
53273,\nVegan Zucchini Bread,"['3 cups all-purpose flour', '1 teaspoon salt'...",['\nPreheat an oven to 325 degrees F (165 degr...,NaN


In [34]:
# Upload the Kaggle dataset
df2=pd.read_csv('/content/core-data_recipe.csv')

In [35]:
# Take a look on the dataset
df2

,recipe_id,recipe_name,image_url,ingredients,cooking_directions,nutritions
0,240488,"Pork Loin, Apples, and Sauerkraut",https://images.media-allrecipes.com/userphotos...,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,218939,Foolproof Rosemary Chicken Wings,https://images.media-allrecipes.com/userphotos...,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,87211,Chicken Pesto Paninis,https://images.media-allrecipes.com/userphotos...,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,245714,Potato Bacon Pizza,https://images.media-allrecipes.com/userphotos...,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,218545,Latin-Inspired Spicy Cream Chicken Stew,https://images.media-allrecipes.com/userphotos...,skinless boneless chicken breast halves^diced ...,{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
...,...,...,...,...,...,...
45625,222886,Grateful Dead Cocktail,https://images.media-allrecipes.com/userphotos...,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45626,25650,Cheese Filling For Pastries,https://images.media-allrecipes.com/userphotos...,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
45627,23544,Peach Smoothie,https://images.media-allrecipes.com/userphotos...,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45628,170710,Double Dare Peaches,https://images.media-allrecipes.com/userphotos...,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."


In [36]:
# Remove unnecessary columns from the Kaggle dataset 
df2=df2.drop(['image_url','recipe_id'], axis=1)

In [37]:
df2

,recipe_name,ingredients,cooking_directions,nutritions
0,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'..."
2,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,Latin-Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves^diced ...,{'directions': u'Prep\n10 m\nCook\n8 h 15 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name..."
...,...,...,...,...
45625,Grateful Dead Cocktail,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45626,Cheese Filling For Pastries,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
45627,Peach Smoothie,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45628,Double Dare Peaches,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."


In [38]:
# Check the null values exist in scrapped dataset
df1.isnull().sum()

recipe_name               0
ingredients               0
cooking_directions        0
nutritions            34348
dtype: int64

In [39]:
# Remove entries with null values 
df1=df1.dropna()

In [61]:
# Check if there is any null value remains

recipe_name           0
ingredients           0
cooking_directions    0
nutritions            0
dtype: int64

In [40]:
# Check the latest number of entries in the dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18927 entries, 0 to 18926
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recipe_name         18927 non-null  object
 1   ingredients         18927 non-null  object
 2   cooking_directions  18927 non-null  object
 3   nutritions          18927 non-null  object
dtypes: object(4)
memory usage: 739.3+ KB


In [43]:
# Concate both datasets
frames = [df1, df2]

# Name the combined dataset as 'Recipe'
Recipe = pd.concat(frames)

In [44]:
# Check for the combined dataset's info
Recipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64557 entries, 0 to 45629
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recipe_name         64557 non-null  object
 1   ingredients         64557 non-null  object
 2   cooking_directions  64557 non-null  object
 3   nutritions          64557 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [45]:
Recipe

,recipe_name,ingredients,cooking_directions,nutritions
0,\nAir Fryer Halloumi Cheese,"['8 ounces halloumi cheese', '2 teaspoons oliv...",['\nPreheat the air fryer to 360 degrees F (18...,"[{'Calories': '66'}, {'Fat': '6g '}, {'Carbs':..."
1,\nAir Fryer Ranch Chicken Bites,"['1 pound boneless, skinless chicken thighs, t...","['\nCombine chicken, avocado oil, white vinega...","[{'Calories': '328'}, {'Fat': '26g '}, {'Carbs..."
2,\nAir Fryer Frozen Chicken Strips,"['25 ounces frozen cooked, breaded crispy chic...",['\nSet the air fryer temperature to 380 degre...,"[{'Calories': '106'}, {'Fat': '2g '}, {'Carbs'..."
3,\nAir Fryer Spicy Onion Rings,"['2 large sweet onions, sliced 1/2 inch thick'...","['\nWhisk together buttermilk, egg, flour, chi...","[{'Calories': '230'}, {'Fat': '4g '}, {'Carbs'..."
4,\nAir Fryer BBQ Chicken Drummies,"['2 pounds chicken drummies', '1 ½ tablespoons...",['\nPreheat an air fryer to 360 degrees F (180...,"[{'Calories': '483'}, {'Fat': '25g '}, {'Carbs..."
...,...,...,...,...
45625,Grateful Dead Cocktail,fluid ounce tequila^fluid ounce vodka^fluid ou...,{'directions': u'Prep\n5 m\nReady In\n5 m\nCom...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45626,Cheese Filling For Pastries,raisins^brandy^cream cheese^white sugar^all-pu...,{'directions': u'Prep\n15 m\nReady In\n15 m\nS...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
45627,Peach Smoothie,sliced peaches drained^scoops vanilla ice crea...,{'directions': u'Prep\n1 m\nReady In\n1 m\nIn ...,"{u'niacin': {u'hasCompleteData': False, u'name..."
45628,Double Dare Peaches,butter^habanero peppers^fresh peaches^brown su...,{'directions': u'Prep\n20 m\nCook\n10 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'..."


In [48]:
# Remove rows with duplicate values using DataFrame.apply() and lambda function 
Recipe = Recipe.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['recipe_name'], keep='first')

In [49]:
# Check dataset's info after removing duplicate values 
Recipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48012 entries, 0 to 45629
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   recipe_name         48012 non-null  object
 1   ingredients         48012 non-null  object
 2   cooking_directions  48012 non-null  object
 3   nutritions          48012 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [51]:
# Convert dataset into csv file
Recipe.to_csv('FinalRecipeData.csv')